# Load Valid Data
For validation data, we will use the 200 train samples. We load it from my 60k dataset [here][1].

[1]: https://www.kaggle.com/datasets/cdeotte/60k-data-with-context-v2

In [23]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

from typing import Optional, Union
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from dataclasses import dataclass
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

VER=12
LOAD_DATASET_FROM_DISK = True
TRAIN_MODEL = True
USE_CONTEXT = True

# TRAIN WITH SUBSET OF MMLU
# NUM_MMLU_TRAIN_SAMPLES = 2_048
# PARAMETER EFFICIENT FINE TUNING
# PEFT REQUIRES 1XP100 GPU NOT 2XT4
USE_PEFT = True
# NUMBER OF LAYERS TO FREEZE 
# DEBERTA LARGE HAS TOTAL OF 24 LAYERS
FREEZE_LAYERS = 16
# BOOLEAN TO FREEZE EMBEDDINGS
FREEZE_EMBEDDINGS = True
# LENGTH OF CONTEXT PLUS QUESTION ANSWER
MAX_INPUT = 256
# HUGGING FACE MODEL
MODEL = 'microsoft/deberta-v3-large'

# MODEL = 'checkpoints_6/checkpoint-1200/'

In [24]:
df_valid = pd.read_csv('dataset/train_with_context2.csv')
if not USE_CONTEXT:
    df_valid['context'] = ''
print('Validation data size:', df_valid.shape )
df_valid.head()

Validation data size: (200, 8)


,prompt,context,A,B,C,D,E,answer
0,Which of the following statements accurately d...,The presence of a clustered thick disk-like co...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,Which of the following is an accurate definiti...,Many of these systems evolve in a self-similar...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,Which of the following statements accurately d...,It is possible that this usage is related with...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,What is the significance of regularization in ...,Renormalization is distinct from regularizatio...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,Which of the following statements accurately d...,Several qualitative observations can be made o...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [25]:
df_train = pd.concat([
    pd.read_csv('dataset/train_cdeotte.csv'), 
    pd.read_csv('dataset/6000_train_examples_with_context.csv', 
                usecols=['prompt', 'context', 'A', 'B', 'C', 'D', 'E', 'answer'])],
                     ignore_index=True,
)                     

if not USE_CONTEXT:
    df_train['context'] = ''

df_train.dropna(inplace=True)
df_train.shape

(13108, 8)

# Data Loader
Code is from Radek's notebook [here][1] with modifications to the tokenization process.

[1]: https://www.kaggle.com/code/radek1/new-dataset-deberta-v3-large-training

In [26]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k,v in option_to_index.items()}

def preprocess(example):
    assert example['context'] is not None, f"Context is None for example {example}"
    assert example['prompt'] is not None, f"Prompt is None for example {example}"
    
    first_sentence = ["[CLS] " + example['context']] * 5
    
    for option in 'ABCDE':
        assert example[option] is not None, f"Option {option} is None for example {example}"
        
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
    
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation='only_first', 
                                  max_length=MAX_INPUT, add_special_tokens=False)
    tokenized_example['label'] = option_to_index[example['answer']]
    
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [27]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
dataset_valid = Dataset.from_pandas(df_valid)
dataset = Dataset.from_pandas(df_train)
#dataset = dataset.remove_columns(["__index_level_0__"])
dataset

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\hzf04d\Anaconda3\envs\cuda2\lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Dataset({
    features: ['prompt', 'context', 'A', 'B', 'C', 'D', 'E', 'answer', '__index_level_0__'],
    num_rows: 13108
})

In [28]:
dataset_valid_path = 'dataset/tokenized_dataset_valid'
dataset_path = 'dataset/tokenized_dataset'

if LOAD_DATASET_FROM_DISK:
    tokenized_dataset_valid = Dataset.from_file(dataset_valid_path + '/data-00000-of-00001.arrow')
    tokenized_dataset = Dataset.from_file(dataset_path + '/data-00000-of-00001.arrow')
else:
    tokenized_dataset_valid = dataset_valid.map(preprocess, remove_columns=df_valid.columns.tolist())
    tokenized_dataset = dataset.map(preprocess, remove_columns=df_train.columns.tolist())
    
    # Save the tokenized datasets
    tokenized_dataset_valid.save_to_disk(dataset_valid_path)
    tokenized_dataset.save_to_disk(dataset_path)
tokenized_dataset

Dataset({
    features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 13108
})

# Build Model
We will use a Hugging Face AutoModelForMultipleChoice. For the list of possible models, see Hugging Face's repository [here][1]. We can optionally use PEFT to accelerate training and use less memory. However i have noticed that validation accuracy is less. (Note that PEFT requires us to use 1xP100 not 2xT4 GPU. I'm not sure why). We can also optionally freeze layers. This also accelerates training and uses less memory. However validation accuracy may become less.

[1]: https://huggingface.co/models

In [29]:
model = AutoModelForMultipleChoice.from_pretrained(MODEL)

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
# # NOTE PEFT REQUIRES US TO USE 1XP100 NOT 2XT4. I'M NOT SURE WHY.
# if USE_PEFT:
#     !pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl

In [31]:
if USE_PEFT:
    print('We are using PEFT.')
    peft_config = LoraConfig(
        r=8, lora_alpha=4, task_type=TaskType.SEQ_CLS, lora_dropout=0.1, 
        bias="none", inference_mode=False, 
        target_modules=["query_proj", "value_proj"],
        modules_to_save=['classifier','pooler'],
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

We are using PEFT.
trainable params: 2887682 || all params: 436899842 || trainable%: 0.6609482820549979


In [32]:
if FREEZE_EMBEDDINGS:
    print('Freezing embeddings.')
    for param in model.deberta.embeddings.parameters():
        param.requires_grad = False
if FREEZE_LAYERS>0:
    print(f'Freezing {FREEZE_LAYERS} layers.')
    for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
        for param in layer.parameters():
            param.requires_grad = False

Freezing embeddings.
Freezing 16 layers.


# MAP@3 Metric
The competition metric is MAP@3 therefore we will make a custom code to add to Hugging Face's trainer. Discussion [here][1]

[1]: https://www.kaggle.com/competitions/kaggle-llm-science-exam/discussion/435602

In [33]:
def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

def compute_metrics(p):
    predictions = p.predictions.tolist()
    labels = p.label_ids.tolist()
    return {"map@3": map_at_3(predictions, labels)}

# Train and Save 
We will now train and save our model using Hugging Face's easy to use trainer. By adjusting the parameters in this notebook, we can achieve `CV MAP@3 = 0.915+` and corresponding single model `LB MAP@3 = 0.830+` wow!

In we run this notebook outside of Kaggle then we can train longer and with more RAM. If we run this notebook on Kaggle, then we need to use tricks to train models efficiently. Here are some ideas:
* use fp16 (this speeds up T4 not P100)
* use gradient_accumlation_steps (this simulates larger batch sizes)
* use gradient_checkpointing (this uses disk to save RAM)
* use 2xT4 instead of 1xP100 (this doubles GPUs)
* freeze model embeddings (this reduces weights to train)
* freeze some model layers (this reduces weights to train)
* use PEFT (this reduces weights to train)
* increase LR and decrease epochs (this reduces work)
* use smaller models (this reduces weights to train)

In [34]:
per_device_train_batch_size = 1
gradient_accumulation_steps = int(8 / per_device_train_batch_size)

training_args = TrainingArguments(
    # resume_from_checkpoint="checkpoints_6/checkpoint-1200/",
    warmup_ratio=0.1, 
    learning_rate=2e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    report_to='none',
    output_dir = f'./checkpoints_{VER}',
    overwrite_output_dir=True,
    fp16=True,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_steps=75,
    evaluation_strategy='steps',
    eval_steps=75,
    save_strategy="steps",
    save_steps=75,
    load_best_model_at_end=False,
    metric_for_best_model='map@3',
    lr_scheduler_type='cosine_with_restarts', # ['linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup', 'inverse_sqrt', 'reduce_lr_on_plateau']
    weight_decay=0.01,
    save_total_limit=2,
)

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset_valid,
    compute_metrics = compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

if TRAIN_MODEL:
    trainer.train()
    trainer.save_model(f'model_v{VER}')

c:\Users\hzf04d\Anaconda3\envs\cuda2\lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


  0%|          | 0/3276 [00:00<?, ?it/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.6178, 'learning_rate': 4.573170731707318e-06, 'epoch': 0.05}


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_loss': 1.07358717918396, 'eval_map@3': 0.7925000000000001, 'eval_runtime': 22.1328, 'eval_samples_per_second': 9.036, 'eval_steps_per_second': 4.518, 'epoch': 1.92}
{'loss': 1.2382, 'learning_rate': 1.6553218908286207e-08, 'epoch': 1.97}


c:\Users\hzf04d\Anaconda3\envs\cuda2\lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_loss': 1.0734565258026123, 'eval_map@3': 0.7925000000000001, 'eval_runtime': 22.5172, 'eval_samples_per_second': 8.882, 'eval_steps_per_second': 4.441, 'epoch': 1.97}
{'train_runtime': 5886.6272, 'train_samples_per_second': 4.453, 'train_steps_per_second': 0.557, 'train_loss': 1.4269542542714921, 'epoch': 2.0}


# Verify Saved Model
During training, we see the MAP@3 validation score above. Let's load the saved model and compute it again here to verify that our model is saved correctly.

In [36]:
del model, trainer

In [37]:
if USE_PEFT:
    model = AutoModelForMultipleChoice.from_pretrained(MODEL)
    model = get_peft_model(model, peft_config)
    checkpoint = torch.load(f'model_v{VER}/adapter_model.bin')
    model.load_state_dict(checkpoint)
else:
    model = AutoModelForMultipleChoice.from_pretrained(f'model_v{VER}')
trainer = Trainer(model=model)

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for PeftModelForSequenceClassification:
	Missing key(s) in state_dict: "base_model.model.deberta.embeddings.word_embeddings.weight", "base_model.model.deberta.embeddings.LayerNorm.weight", "base_model.model.deberta.embeddings.LayerNorm.bias", "base_model.model.deberta.encoder.layer.0.attention.self.query_proj.weight", "base_model.model.deberta.encoder.layer.0.attention.self.query_proj.bias", "base_model.model.deberta.encoder.layer.0.attention.self.query_proj.lora_A.default.weight", "base_model.model.deberta.encoder.layer.0.attention.self.query_proj.lora_B.default.weight", "base_model.model.deberta.encoder.layer.0.attention.self.key_proj.weight", "base_model.model.deberta.encoder.layer.0.attention.self.key_proj.bias", "base_model.model.deberta.encoder.layer.0.attention.self.value_proj.weight", "base_model.model.deberta.encoder.layer.0.attention.self.value_proj.bias", "base_model.model.deberta.encoder.layer.0.attention.self.value_proj.lora_A.d

In [ ]:
test_df = pd.read_csv('dataset/train_with_context2.csv')
tokenized_test_dataset = Dataset.from_pandas(test_df).map(
        preprocess, remove_columns=['prompt', 'context', 'A', 'B', 'C', 'D', 'E'])

test_predictions = trainer.predict(tokenized_test_dataset).predictions
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]

c:\Users\hzf04d\Anaconda3\envs\cuda2\lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

  0%|          | 0/25 [00:00<?, ?it/s]

# Compute Validation Score

In [ ]:
# https://www.kaggle.com/code/philippsinger/h2ogpt-perplexity-ranking
import numpy as np
def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u].split()
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3 / U

In [ ]:
m = MAP_at_3(test_df.prediction.values, test_df.answer.values)
print( 'CV MAP@3 =',m )

CV MAP@3 = 0.8533333333333334


In [ ]:
# Save parameters, constants and MAP@3 score to config{SIMULATION}/ folder
import json
with open(f'config{VER}.json', 'w') as f:
    json.dump({
        'SIMULATION': VER,
        'LOAD_DATASET_FROM_DISK': LOAD_DATASET_FROM_DISK,
        'USE_PEFT': USE_PEFT,
        'FREEZE_LAYERS': FREEZE_LAYERS,
        'FREEZE_EMBEDDINGS': FREEZE_EMBEDDINGS,
        'MAX_INPUT': MAX_INPUT,
        'MODEL': MODEL,
        'MAP@3 test': m,
        'training args': training_args.to_dict(),
    }, f, indent=4)

# Analyse erros

In [ ]:
import textwrap

def printer(text, width=150):
    text_lines = textwrap.wrap(text, width=width)
    for line in text_lines:
        print(f"\t{line}")
              
def print_row(row):
    to_print_part1 = row['context']
    to_print_part2 = 'Prompt:\n' + '\t' + row['prompt']
    to_print_part3 = 'Answers:\n'
    for col in 'ABCDE':
        to_print_part3 += f"\t- {row[col]}\n"
    to_print_part4 = f"True Answer: {row['answer']}\n"
    # remaining_width = MAX_INPUT - len(to_print_part2) - len(to_print_part3) - len(to_print_part4) - len('Prompt:Answers:True Answer: ')
    context = textwrap.shorten(to_print_part1, width=1000, placeholder="...")
    
    print("Context:")
    printer(context)
    print(to_print_part2)
    print(to_print_part3)
    print(to_print_part4)
    print()
    
_ = test_df[test_df['prediction'].str[0] != test_df['answer']].apply(print_row, axis=1)

Context:
	It is possible that this usage is related with the Greek name of the island of Sicily, Trinacria (Τρινακρία "having three headlands").Liddell and
	Scott’s Greek-English Lexicon (A Lexicon Abridged from), Oxford, 1944, p.27, Cassell's Latin Dictionary, Marchant, J.R.V, & Charles, Joseph F.,
	(Eds.), Revised Edition, 1928 The Sicilian triskeles is shown with the head of Medusa at the center.Matthews, Jeff (2005) Symbols of Naples The
	ancient symbol has been re-introduced in modern flags of Sicily since 1848. An early flag of Sicily, proposed in 1848, included the Sicilian triskeles
	or "Trinacria symbol". The triskeles was adopted as emblem by the rulers of Syracuse. The oldest find of a triskeles in Sicily is a vase dated to 700
	BCE, for which researchers assume a Minoan-Mycenaean origin. ===Roman period and Late Antiquity=== Late examples of the triple spiral symbols are
	found in Iron Age Europe, e.g. carved in rock in Castro Culture settlement in Galicia, Asturias and Nor

In [ ]:
test_df[test_df['prediction'].str[0] != test_df['answer']]

,prompt,context,A,B,C,D,E,answer,prediction
2,Which of the following statements accurately d...,It is possible that this usage is related with...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A,D A C
5,Which of the following statements accurately d...,"For a closed Gaussian surface, electric flux i...",Gauss's law holds only for situations involvin...,"Gauss's law holds in all cases, but it is most...","Gauss's law, which applies equally to all elec...",Gauss's law only holds for electric fields wit...,"Gauss's law, which holds for all situations, i...",B,C B E
13,What is the Roche limit?,"In celestial mechanics, the Roche limit, also ...",The Roche limit is the distance at which tidal...,The Roche limit is the distance at which tidal...,The Roche limit is the distance at which tidal...,The Roche limit is the distance at which tidal...,The Roche limit is the distance at which tidal...,D,E D C
17,What is the butterfly effect?,The butterfly effect describes a phenomenon in...,The butterfly effect is a physical cause that ...,The butterfly effect is a distributed causalit...,The butterfly effect is a proportionality betw...,The butterfly effect is a small push that is n...,The butterfly effect is a phenomenon that high...,E,D E B
23,What did Newton's manuscripts of the 1660s show?,Newton was well-versed in both classics and mo...,Newton learned about tangential motion and rad...,Newton's manuscripts did not show any evidence...,Newton combined tangential motion with the eff...,Newton's manuscripts showed that he learned ab...,Newton's manuscripts showed that he was indebt...,C,D C A
24,What is the decay energy for the free neutron ...,The following diagram gives a summary sketch o...,0.013343 MeV,0.013 MeV,"1,000 MeV",0.782 MeV,0.782343 MeV,E,D E A
33,What are amorphous ferromagnetic metallic allo...,Amorphous metals can be grouped in two categor...,Amorphous ferromagnetic metallic alloys are cr...,Amorphous ferromagnetic metallic alloys are no...,Amorphous ferromagnetic metallic alloys are cr...,Amorphous ferromagnetic metallic alloys are no...,Amorphous ferromagnetic metallic alloys are no...,D,E D B
34,What is the Penrose process?,The Penrose process (also called Penrose mecha...,The Penrose process is a mechanism through whi...,The Penrose process is a mechanism through whi...,The Penrose process is a mechanism through whi...,The Penrose process is a mechanism through whi...,The Penrose process is a mechanism through whi...,C,B C E
35,What was the aim of the Gravity Probe B (GP-B)...,Gravity Probe B (GP-B) was a satellite-based e...,To prove that pressure contributes equally to ...,"To measure spacetime curvature near Earth, wit...",To measure the distribution of Fe and Al on th...,To confirm the relatively large geodetic effec...,To measure the discrepancy between active and ...,B,D E B
38,What is the spin quantum number?,"In physics, the spin quantum number is a quant...",The spin quantum number is a measure of the di...,The spin quantum number is a measure of the si...,The spin quantum number is a measure of the ch...,The spin quantum number is a measure of the sp...,The spin quantum number is a dimensionless qua...,E,D E C


# Submission

In [ ]:
predictions_as_string

['D C B',
 'A D E',
 'D A C',
 'C B D',
 'D E A',
 'C B E',
 'A D B',
 'D E B',
 'C B A',
 'A E C',
 'E D A',
 'A B E',
 'C A E',
 'E D C',
 'B A D',
 'B D E',
 'E A C',
 'D E B',
 'A B C',
 'E B D',
 'D B C',
 'D C E',
 'C D A',
 'D C A',
 'D E A',
 'E D A',
 'A C E',
 'D C B',
 'E B C',
 'C B A',
 'B A E',
 'E C A',
 'E B A',
 'E D B',
 'B C E',
 'D E B',
 'E B D',
 'A C D',
 'D E C',
 'A E D',
 'E D A',
 'A B D',
 'D C E',
 'D B C',
 'E C D',
 'A E B',
 'B E C',
 'C E B',
 'A D E',
 'B A C',
 'B D E',
 'E D C',
 'B C A',
 'D C A',
 'B D A',
 'B D E',
 'C A E',
 'C B A',
 'D E C',
 'A B E',
 'B E D',
 'E D B',
 'C A B',
 'A E C',
 'A B D',
 'C D E',
 'C A D',
 'E A B',
 'A D C',
 'D E A',
 'C A B',
 'A D E',
 'C A D',
 'B E D',
 'D A E',
 'A C B',
 'D B E',
 'B C D',
 'C E B',
 'E A C',
 'C E B',
 'A D E',
 'B D C',
 'A D C',
 'E C D',
 'C E D',
 'D A C',
 'B A E',
 'E C A',
 'D E C',
 'B D E',
 'B D C',
 'B E C',
 'E B D',
 'E A B',
 'C D A',
 'C D B',
 'B D E',
 'D C B',
 'D C E',
